## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-12-19-20-52-26 +0000,nypost,"Muhammad Aziz Umurzakov, teen killed in Brown ...",https://nypost.com/2025/12/19/us-news/muhammad...
1,2025-12-19-20-51-02 +0000,nyt,Can This Man Finally Defeat Mexico’s Cartels?,https://www.nytimes.com/2025/12/19/world/ameri...
2,2025-12-19-20-48-44 +0000,bbc,Removed Ben & Jerry's chair says Magnum aimed ...,https://www.bbc.com/news/articles/cn98pzr8x79o...
3,2025-12-19-20-47-48 +0000,nyt,Trump Administration Pauses Green Card Lottery...,https://www.nytimes.com/2025/12/19/us/brown-sh...
4,2025-12-19-20-44-30 +0000,startribune,Vikings-Giants preview: Can Minnesota extend i...,https://www.startribune.com/minnesota-vikings-...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2357/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
26,trump,44
79,new,21
91,ukraine,17
188,bondi,17
6,shooting,16


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
329,2025-12-18-23-26-52 +0000,nypost,GOP senators push Trump to use $5 billion in R...,https://nypost.com/2025/12/18/world-news/gop-s...,118
345,2025-12-18-22-28-39 +0000,nypost,President Trump gives federal workers 2 new ho...,https://nypost.com/2025/12/18/us-news/presiden...,92
13,2025-12-19-20-31-32 +0000,nypost,Trump says he will haul in ‘rich’ health insur...,https://nypost.com/2025/12/19/us-news/trump-sa...,89
3,2025-12-19-20-47-48 +0000,nyt,Trump Administration Pauses Green Card Lottery...,https://www.nytimes.com/2025/12/19/us/brown-sh...,88
278,2025-12-19-03-19-00 +0000,wsj,President Trump signed into law new powers to ...,https://www.wsj.com/politics/national-security...,88


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
329,118,2025-12-18-23-26-52 +0000,nypost,GOP senators push Trump to use $5 billion in R...,https://nypost.com/2025/12/18/world-news/gop-s...
256,81,2025-12-19-05-54-00 +0000,wsj,Brown and MIT Shooting Suspect Found Dead in N...,https://www.wsj.com/us-news/brown-shooting-sus...
56,58,2025-12-19-18-52-48 +0000,nypost,NASCAR legend Greg Biffle’s plane was in air o...,https://nypost.com/2025/12/19/sports/nascar-le...
244,55,2025-12-19-08-34-47 +0000,nyt,Australia Announces Plan to Buy Back Guns in W...,https://www.nytimes.com/2025/12/19/world/asia/...
13,45,2025-12-19-20-31-32 +0000,nypost,Trump says he will haul in ‘rich’ health insur...,https://nypost.com/2025/12/19/us-news/trump-sa...
302,42,2025-12-19-00-54-18 +0000,nypost,Erika Kirk vows to get JD Vance elected presid...,https://nypost.com/2025/12/18/us-news/erika-ki...
155,42,2025-12-19-15-14-00 +0000,wsj,Home sales rose in November for the third stra...,https://www.wsj.com/economy/housing/home-sales...
57,41,2025-12-19-18-51-36 +0000,latimes,Justice Department won't meet Friday deadline ...,https://www.latimes.com/politics/story/2025-12...
319,39,2025-12-18-23-46-10 +0000,nypost,TikTok’s Chinese owner ByteDance inks deal to ...,https://nypost.com/2025/12/18/business/tiktoks...
260,39,2025-12-19-05-12-07 +0000,nypost,Wisconsin Judge Hannah Dugan guilty of obstruc...,https://nypost.com/2025/12/19/us-news/judge-ha...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
